## Logistic Regression Approach

1. Convert business problem to data science problem.
2. Load data.
3. Understand the data.
4. Data preprocessing.
5. Exploratory data analysis(EDA).
6. Model building.
7. Model diagnostics.
8. Predictions and Evaluations.

### 1. Problem Statement

Build a classification engine which classifies a customer if he/she has churned out of the bank or not, based on various features like credit score, balance, tenure, gender etc.

In [1]:
# import necessary modules
import pandas as pd
import numpy as np

pd.set_option('display.max_rows', 800)
pd.set_option('display.max_columns', 500)

import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
sns.set()

import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

### 2. Load Data

In [2]:
churn_data = pd.read_csv('data.csv', index_col='RowNumber')

### 3. Understanding the data

In [3]:
churn_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 1 to 10000
Data columns (total 13 columns):
CustomerId         10000 non-null int64
Surname            10000 non-null object
CreditScore        10000 non-null int64
Geography          10000 non-null object
Gender             10000 non-null object
Age                10000 non-null int64
Tenure             10000 non-null int64
Balance            10000 non-null float64
NumOfProducts      10000 non-null int64
HasCrCard          10000 non-null int64
IsActiveMember     10000 non-null int64
EstimatedSalary    10000 non-null float64
Exited             10000 non-null int64
dtypes: float64(2), int64(8), object(3)
memory usage: 1.1+ MB


In [4]:
churn_data.describe()

,CustomerId,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
count,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000
mean,1.569094e+07,650.528800,38.921800,5.012800,76485.889288,1.530200,0.70550,0.515100,100090.239881,0.203700
std,7.193619e+04,96.653299,10.487806,2.892174,62397.405202,0.581654,0.45584,0.499797,57510.492818,0.402769
min,1.556570e+07,350.000000,18.000000,0.000000,0.000000,1.000000,0.00000,0.000000,11.580000,0.000000
25%,1.562853e+07,584.000000,32.000000,3.000000,0.000000,1.000000,0.00000,0.000000,51002.110000,0.000000
50%,1.569074e+07,652.000000,37.000000,5.000000,97198.540000,1.000000,1.00000,1.000000,100193.915000,0.000000
75%,1.575323e+07,718.000000,44.000000,7.000000,127644.240000,2.000000,1.00000,1.000000,149388.247500,0.000000
max,1.581569e+07,850.000000,92.000000,10.000000,250898.090000,4.000000,1.00000,1.000000,199992.480000,1.000000


In [5]:
churn_data.head()

,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
RowNumber,,,,,,,,,,,,,
1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [6]:
num_cols = churn_data.select_dtypes(include=np.number).columns
cat_cols = churn_data.select_dtypes(exclude=np.number).columns

print(f"Numerical columns: {num_cols.values}\n")
print(f"Categorical columns: {cat_cols.values}\n")

Numerical columns: ['CustomerId' 'CreditScore' 'Age' 'Tenure' 'Balance' 'NumOfProducts'
 'HasCrCard' 'IsActiveMember' 'EstimatedSalary' 'Exited']

Categorical columns: ['Surname' 'Geography' 'Gender']



### 4. Data pre-processing

In [7]:
# Let's do one hot encoding for the column `Geography` as Logistic Regression model only accepts numerical values
churn_data_new = pd.get_dummies(prefix='Geo', data=churn_data, columns=['Geography'])
churn_data_new.head()

,CustomerId,Surname,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geo_France,Geo_Germany,Geo_Spain
RowNumber,,,,,,,,,,,,,,,
1,15634602,Hargrave,619,Female,42,2,0.00,1,1,1,101348.88,1,1,0,0
2,15647311,Hill,608,Female,41,1,83807.86,1,0,1,112542.58,0,0,0,1
3,15619304,Onio,502,Female,42,8,159660.80,3,1,0,113931.57,1,1,0,0
4,15701354,Boni,699,Female,39,1,0.00,2,0,0,93826.63,0,1,0,0
5,15737888,Mitchell,850,Female,43,2,125510.82,1,1,1,79084.10,0,0,0,1


In [14]:
# For `Gender` column, we will do Label Encoding, 1: Male, 0: Female
churn_data_new['Gender'] = churn_data_new['Gender'].map({'Male':1, 'Female':0})

In [19]:
churn_data_new.head()

,CustomerId,Surname,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geo_France,Geo_Germany,Geo_Spain
RowNumber,,,,,,,,,,,,,,,
1,15634602,Hargrave,619,0,42,2,0.00,1,1,1,101348.88,1,1,0,0
2,15647311,Hill,608,0,41,1,83807.86,1,0,1,112542.58,0,0,0,1
3,15619304,Onio,502,0,42,8,159660.80,3,1,0,113931.57,1,1,0,0
4,15701354,Boni,699,0,39,1,0.00,2,0,0,93826.63,0,1,0,0
5,15737888,Mitchell,850,0,43,2,125510.82,1,1,1,79084.10,0,0,0,1


### 5. Exploratory Data Analysis (EDA)